In [75]:
import numpy as np

np.random.seed(42)

def frob(A, B):
    return np.trace(A.T @ B)

def symmetric(n):
    A = np.random.normal(size=(n, n))
    return (A + A.T) / 2

def positive_definite(n):
    A = np.random.normal(size=(n, n))
    assert np.all(np.linalg.eigvals(A @ A.T)) > 0
    return A @ A.T

class GradChecker:
    def __init__(self, f, grad, grad2, bilinear_d2f=False):
        self.f = f
        self.grad = grad
        self.grad2 = grad2
        self.bilinear_d2f = bilinear_d2f
    
    def estimate_grad(self, x, d, eps):
        if not isinstance(x, np.ndarray): # R -> R
            return ((self.f(x + eps) - self.f(x)) / eps)
        return ((self.f(x + d * eps) - self.f(x)) / eps).item()
    
    def calculate_grad(self, x, d):
        if not isinstance(x, np.ndarray): # R -> R
            return self.grad(x)
        elif x.shape[1] == 1: # R^n - > R
            return (self.grad(x).T @ d).item() 
        return frob(self.grad(x), d) # R^{nxn} -> R
             
    def estimate_grad2(self, x, d, eps):
        if not isinstance(x, np.ndarray): # R -> R
            return ((self.grad(x + eps) - self.grad(x)) / eps)
        return (self.calculate_grad(x + d * eps, d) - self.calculate_grad(x, d)) / eps

    def calculate_grad2(self, x, d):
        if not isinstance(x, np.ndarray): # R -> R
            return self.grad2(x)
        elif not self.bilinear_d2f: # R^n -> R
            return (d.T @ (self.grad2(x) @ d)).item()
        return self.grad2(x, d) # calc d2f as a bilinear form
    
    def check(self, x, d, eps, tol=1e-5, verbose=False):
        df_est = self.estimate_grad(x, d, eps)
        df = self.calculate_grad(x, d)
        df2_est = self.estimate_grad2(x, d, eps)
        df2 = self.calculate_grad2(x, d)
        
        if verbose:
            print("~dF(x)[d] = {:.5f}".format(df_est))
            print("dF(x)[d] = {:.5f}".format(df))
            print("~d2F(x)[d,d] = {:.5f}".format(df2_est))
            print("d2F(x)[d,d] = {:.5f}".format(df2))
        
        assert np.abs(df_est - df) <= tol, "First differential estimation doesn't correspond to true one"
        assert np.abs(df2_est - df2) <= tol, "Second differential estimation doesn't correspond to true one"

In [76]:
#n = 3
#A = np.diag([1, 1, 0])
#b = np.ones(n).reshape(-1, 1)
#c = -10

def test_1a():
    def f(t):
        x = np.linalg.inv(A + t * np.eye(n)) @ b
        return x.T @ x

    def df(t):
        return -2 * b.T @ np.linalg.matrix_power(np.linalg.inv(A + t * np.eye(n)), 3) @ b

    def d2f(t):
        return 6 * b.T @ np.linalg.matrix_power(np.linalg.inv(A + t * np.eye(n)), 4) @ b

    checker = GradChecker(f, df, d2f)

    n = 3
    A = positive_definite(n)
    b = np.array([1,2,3])
    eps = 1e-7
    t = 1
    dt = 42

    checker.check(t, dt, eps)
    print("1a OK")

In [82]:
def test_1b():
    def f(x):
        return 0.5 * np.linalg.norm(x @ x.T - A) ** 2

    def df(x):
        return 2 * ((x.T @ x) * x - A @ x)

    def d2f(x):
        return 2 * (x.T @ x * np.eye(n) + 2 * x @ x.T - A)

    checker = GradChecker(f, df, d2f)
    
    n = 3
    A = positive_definite(n)
    eps = 1e-7
    x = np.random.normal(size=(n,1))
    d = np.random.normal(size=(n,1))

    checker.check(x, d, eps)
    print("1b OK")

In [84]:
def test_1c():
    def f(x):
        return np.exp((x.T @ x) * np.log((x.T @ x)))

    def df(x):
        return 2 * f(x) * (np.log(x.T @ x) + 1) * x

    def d2f(x):
        return 2 * f(x) * (2 * (np.log(x.T @ x) + 1)**2 * x @ x.T + 2 / (x.T @ x) * x @ x.T + (np.log(x.T @ x) + 1) * np.eye(n))

    checker = GradChecker(f, df, d2f)
    
    n = 3
    A = positive_definite(n)
    eps = 1e-7
    x = np.random.normal(size=(n,1))
    d = np.random.normal(size=(n,1))

    checker.check(x, d, eps)
    print("1c OK")

In [85]:
test_1a()
test_1b()
test_1c()

1a OK
1b OK
1c OK
